In [27]:
# import necessary libaries and data files
import numpy as np
from data import datas
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


#load data and manupulate datas if needed
bus_data, line_data, load_data, gen_data, wind_data = datas() #data loading

line_data = pd.DataFrame(line_data).sort_values(by=[0,1]).values #line data sorting and data type change

bus_data = pd.DataFrame(bus_data).sort_values(by = [0]).values #Bus data sorting and data type change

load_data = pd.DataFrame(load_data).sort_values(by = [0]).astype({0:'int'}).set_index(0) #Load data sorting and data type change

gen_data = pd.DataFrame(gen_data).sort_values(by = [0]).astype({0:'int'}).set_index(0) #Generation data sorting and data type change

wind_data = pd.DataFrame(wind_data).sort_values(by = [0]).astype({0:'int'}).set_index(0) #Wind Generation data sorting and data type change


base_mva = 100
R = line_data[:, 2]
X = line_data[:, 3]
B = 0.5j * line_data[:, 4]
taps = line_data[:,5]
Bsh = 1j* bus_data[:, 3]
Z = R + 1j * X
Y = 1 / Z
nline = len(line_data[:, 0])
nbus = int(np.amax(line_data[:, :2]))  # Total Number of buses



In [28]:
def tap_handle(x):
    '''Function to handle transformer tap ratio'''
    if np.real(x) == 0:
        return 1
    else:
        return complex(x)
    

# Y Bus Formation
def Y_bus():
    '''Function to create Y Bus matrix '''
    Ybus = np.zeros((nbus, nbus), dtype=complex)
    Y1 = np.zeros((73, 73), dtype=complex)
    np.fill_diagonal(Y1,Bsh)
    
    for k in range(nline):
        # Off Diagonal Elements
        Ybus[int(line_data[k, 0]) - 1, int(line_data[k, 1]) - 1] -= (Y[k])/np.conj(tap_handle(taps[k]))
        Ybus[int(line_data[k, 1]) - 1, int(line_data[k, 0]) - 1] -= (Y[k])/tap_handle(taps[k])

        # Diagonal Elements
        Ybus[int(line_data[k, 0]) - 1, int(line_data[k, 0]) - 1] += (Y[k] + B[k])/pow(abs(tap_handle(taps[k])),2)
        Ybus[int(line_data[k, 1]) - 1, int(line_data[k, 1]) - 1] += Y[k] + B[k]
    
    Y_bus_ = Ybus + Y1
    return Y_bus_

In [29]:
#Data from Bus_data
bus_no = bus_data[:, 0].astype(int)
bus_type = bus_data[:,1].astype(int)

#Data from Load_data
pl = np.zeros(nbus)
pf = np.zeros(nbus)
delta = np.zeros(nbus)
for k in load_data.index:
    pl[k-1] = load_data.loc[k, 2]/base_mva #base power
    pf[k-1] = load_data.loc[k, 1]
ql = pl * np.tan(np.arccos(pf))

#From Generation Data
pg_ = np.zeros(nbus)
qg = np.zeros(nbus)
vmag = np.ones(nbus)
qmin = np.zeros(nbus)
qmax = np.zeros(nbus)
for k in gen_data.index:
    pg_[k-1] = (gen_data.loc[k, 4]).sum()/base_mva
    vmag[k-1] = np.mean(gen_data.loc[k, 3])
    qmin[k-1] = gen_data.loc[k, 2].sum()
    qmax[k-1] = gen_data.loc[k, 1].sum()

# From Wind Data
pwg = np.zeros(nbus)
for k in wind_data.index:
    pwg[k-1] = (wind_data.loc[k, 4])/base_mva

pg = pg_ + pwg
b_data = pd.DataFrame()
b_data['Bus_No'] = bus_no
b_data['Bus_type'] = bus_type
b_data['Pg'] = pg
b_data['Qg'] = qg
b_data['Pl'] = pl
b_data['Ql'] = ql
b_data['Vmag'] = vmag
b_data['Delta'] = delta
b_data['Qmin'] = qmin/base_mva
b_data['Qmax'] = qmax/base_mva
b_data["P_sh"] = b_data["Pg"] - b_data["Pl"]
b_data["Q_sh"] = b_data["Qg"] - b_data["Ql"]
b_data = b_data.set_index('Bus_No')

In [30]:
def f_x(df_, y_bus):
    '''Function to calculate real and reactive power'''
    df = df_.copy(deep=True)
    P = np.zeros(nbus)
    Q = np.zeros(nbus)
    Y = np.absolute(y_bus)
    th = np.angle(y_bus, deg=False)
    for i in df.index:
        for j in df.index:
            P[i-1] += (df.loc[i, "Vmag"] * df.loc[j, "Vmag"] * Y[i-1, j-1] * 
                       np.cos(th[i-1, j-1] - df.loc[i, 'Delta'] + df.loc[j, 'Delta'] ))
            Q[i-1] -= (df.loc[i, "Vmag"] * df.loc[j, "Vmag"] * Y[i-1, j-1] * 
                       np.sin(th[i-1, j-1] - df.loc[i, 'Delta'] + df.loc[j, 'Delta'] ))

    return P, Q        


In [31]:
def delta_f_x(df_, y_bus):
    '''Function to calculate power mismatch'''
    P, Q = f_x(df_, y_bus)
    df_['P_calc'] = P
    df_['Q_calc'] = Q
    df_['D_P'] = df_['P_sh'] - df_['P_calc']
    df_['D_Q'] = df_['Q_sh'] - df_['Q_calc']

    return df_

In [32]:
def Jacobian(df, y_bus):
    '''Function to create Jacobian Matrix,
        Create reduced jacobian'''
    df_ = delta_f_x(df, y_bus)
    B = np.imag(y_bus)

    #J1
    J1 = np.zeros((nbus, nbus))
    for i in df_.index:
        for j in df_.index:
            if i == j:
                J1[i-1, i-1] = -(df_.loc[i, 'Vmag'] * B[i-1, i-1]) #Diagonal element
            else:
                J1[i-1, j-1] = -(df_.loc[i, 'Vmag'] * B[i-1, j-1]) # Off Diagonal Element    
    
                
    #J4
    J4 = np.zeros((nbus, nbus))
    for i in df_.index:
        for j in df_.index:
            if i == j:
                J4[i-1, i-1] = -(df_.loc[i, 'Vmag'] * B[i-1, i-1]) #Diagonal element
            else:
                J4[i-1, j-1] = -(df_.loc[i, 'Vmag'] * B[i-1, j-1]) # Off Diagonal Element 
                
    #Reducing Jacobian
    J11 = J1[np.where(df_['Bus_type'] != 3)[0], :][:, np.where(df_['Bus_type'] != 3)[0]]
    J44 = J4[np.where(df_['Bus_type'] == 1)[0], :][:, np.where(df_['Bus_type'] == 1)[0]]

    #Reducing D_P and D_Q %% Determing indices
    d_p = df_.loc[(np.where(df_['Bus_type'] != 3)[0] + 1)]['D_P']
    d_q = df_.loc[(np.where(df_['Bus_type'] == 1)[0] + 1)]['D_Q']
    
    return d_p, d_q, J11, J44
    

In [33]:
'''def N_R(df, iter, tol):
    "Function to perform Newton Raphson for solving non linear equations"
    b_data_ = df.copy(deep=True)
    for i in range(iter):
        D_P, D_Q, J1, J4 = Jacobian(b_data, Y_bus())
        del_delta = np.linalg.solve(J1, D_P)
        print(max(np.abs(del_delta)))
        if max(np.abs(del_delta)) <= tol:
            print('aa')
            pass   
        b_data_.loc[D_P.index, 'Delta'] = b_data_.loc[D_P.index, 'Delta'].values + del_delta

    for i in range(iter):
        D_P, D_Q, J1, J4 = Jacobian(b_data, Y_bus())
        del_V = np.linalg.solve(J4, D_Q) 
        
        if max(np.abs(del_V)) <= tol:
            return b_data_
        b_data_.loc[D_Q.index, 'Vmag'] = b_data_.loc[D_Q.index, 'Vmag'].values + del_V
    return "Value Error" '''
    

'def N_R(df, iter, tol):\n    "Function to perform Newton Raphson for solving non linear equations"\n    b_data_ = df.copy(deep=True)\n    for i in range(iter):\n        D_P, D_Q, J1, J4 = Jacobian(b_data, Y_bus())\n        del_delta = np.linalg.solve(J1, D_P)\n        print(max(np.abs(del_delta)))\n        if max(np.abs(del_delta)) <= tol:\n            print(\'aa\')\n            pass   \n        b_data_.loc[D_P.index, \'Delta\'] = b_data_.loc[D_P.index, \'Delta\'].values + del_delta\n\n    for i in range(iter):\n        D_P, D_Q, J1, J4 = Jacobian(b_data, Y_bus())\n        del_V = np.linalg.solve(J4, D_Q) \n        \n        if max(np.abs(del_V)) <= tol:\n            return b_data_\n        b_data_.loc[D_Q.index, \'Vmag\'] = b_data_.loc[D_Q.index, \'Vmag\'].values + del_V\n    return "Value Error" '

In [34]:
import concurrent.futures

def N_R_P(df, iter, tol):
    b_data_ = df.copy(deep=True)
    for i in range(iter):
        D_P, D_Q, J1, J4 = Jacobian(b_data, Y_bus())
        del_delta = np.linalg.solve(J1, D_P)
        #print('Delta',max(np.abs(del_delta)))
        if max(np.abs(del_delta)) <= tol:
            return b_data_['Delta'] 
        elif i == iter-1:
            return b_data_['Delta']
        b_data_.loc[D_P.index, 'Delta'] = b_data_.loc[D_P.index, 'Delta'].values + del_delta
    

def N_R_Q(df, iter, tol):
    b_data_ = df.copy(deep=True)
    for i in range(iter):
        D_P, D_Q, J1, J4 = Jacobian(b_data, Y_bus())
        del_V = np.linalg.solve(J4, D_Q)
        #print('V:',max(np.abs(del_V)))
        if max(np.abs(del_V)) <= tol:
            return b_data_['Vmag']
        elif i == iter-1:
            return b_data_['Vmag']
        b_data_.loc[D_Q.index, 'Vmag'] = b_data_.loc[D_Q.index, 'Vmag'].values + del_V

def main():

    # Using ThreadPoolExecutor for parallel execution
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit the sum and product functions to run in parallel
        P_ = executor.submit(N_R_P, b_data, 50, 0.001)
        Q_ = executor.submit(N_R_Q, b_data, 50, 0.001)

        # Retrieve the results when both calculations are done
        d = P_.result()
        v = Q_.result()

    # Print the results
    #print(d)
    #print(v)
    b_data['Delta'] = np.radians(d)
    b_data['Vmag'] = v

    print(b_data)

if __name__ == "__main__":
    main()


        Bus_type      Pg   Qg      Pl        Ql      Vmag     Delta  Qmin  \
Bus_No                                                                      
1              2  0.5282  0.0  0.5635  0.114718  1.035000 -0.559495 -0.50   
2              1  0.0000  0.0  0.5041  0.103934  1.076187 -0.566325 -0.50   
3              1  0.0000  0.0  0.9342  0.192128  2.200108 -0.401503  0.00   
4              1  0.0000  0.0  0.3855  0.078077  2.467975 -0.541046  0.00   
5              1  0.0000  0.0  0.3707  0.073113  2.248638 -0.553521  0.00   
...          ...     ...  ...     ...       ...       ...       ...   ...   
69             2  4.0000  0.0  0.0000  0.000000  1.050000 -0.506709 -0.50   
70             1  0.0000  0.0  0.0000  0.000000  1.188369 -0.519178 -0.60   
71             2  0.5424  0.0  0.0000  0.000000  1.050000 -0.413922 -1.25   
72             1  0.0000  0.0  0.0000  0.000000  1.736205 -0.573560  0.00   
73             1  0.0000  0.0  0.0000  0.000000  3.312952 -0.449493  0.00   